<a href="https://colab.research.google.com/github/hartmann-pereira/D/blob/main/Pyspark_exploratory_analisys_for_a_web_log_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Spark Example

Notebook only for experimental use, built in pyspark.

This notebook exemplifies the execution of a Spark program in Python.
In this example, spark runs in standalone mode and reads data from the local filesystem, while in cluster mode data is read typically from HDFS dsitributed file system.

Data came from a txt file with web log files.
This was purely made to train myself to code in pyspark as this is a usefull skill to have, also wanted to show a bit of analisys that can be done with this tool. 

A lot of data printing would not be made if this was made with a bigger file, but for experimental purposes and because i knew how big the dataset is its fine.

Spark documentation available at:
https://spark.apache.org/docs/2.3.1/


## Web log analisys example
Firstly we are going to import all the dependencies necessary to run pyspark in a google colab environment. 

Brief index of what the notebook contains. More to come in a near future...

+ We are going to manipulate a web log dataset.

  + Return the list of source ip adresses
  + Find log entries that report acces to a given url
  + Create an inverted index url -> list of unique source ip adresses
  + Count the number of unique ip adresses involved in the attack.
  + and much more...!

In [ ]:
# instalation and imports of all the necessary dependencies, had to copy a lot from a google colab page
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
# configure the variables of the environment 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# enable pyspark to be imported 
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

import pyspark
from operator import add as sum


In [ ]:
!wget -O web_log.txt https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0

--2021-11-03 20:12:09--  https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0r8902uj9yum7dg/web.log [following]
--2021-11-03 20:12:10--  https://www.dropbox.com/s/raw/0r8902uj9yum7dg/web.log
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1ee7ec1446ddbc114ef780c747.dl.dropboxusercontent.com/cd/0/inline/BZTP7tMbvXw6x68G2QFEg3w9oiVse1uSM-ojqDz15nnZzG1q7WhAGCgQJ26nNFDqw1vlOnko9lGdMcDy9TwnBDg-AtLDSNPjsB01_22mbnJ7PXEF4ibRHS25iZv6Jo0Qn7V2pkgxnLirm7xJtxJgJgiA/file# [following]
--2021-11-03 20:12:10--  https://uc1ee7ec1446ddbc114ef780c747.dl.dropboxusercontent.com/cd/0/inline/BZTP7tMbvXw6x68G2QFEg3w9oiVse1uSM-ojqDz15nnZzG1q7WhAGCgQJ26nNFDqw1vlOnko9lGdMcDy9TwnBDg-AtLDSNPjs

In [ ]:
sc = pyspark.SparkContext('local[*]') #initialized a spark context, remember than your code stops the spark context, so before running you have to run this code again

Exploratory analisys of web log data: 

In [ ]:
try:
  lines = sc.textFile('web_log.txt') # here we get the web log document into out spark context to manipulate as we want
  non_empty_lines = lines.filter(lambda line: len(line) > 0) #data cleaning: getting an rdd with all our info
  ips = non_empty_lines.map(lambda line:line.split(" ")[1]) #data cleaning: our data has a lot of info but we want an rdd with only the ip adresses
  distinct_ips = ips.distinct() # data cleaning: since it is a web log entry we have a lot of duplication, distinct ips are only one occurence per ip
  for v in distinct_ips.collect():       # here we just want to output the distinct ips rdd
    print(v) 
  sc.stop()
except:
  sc.stop()

In [ ]:
try:
  lines = sc.textFile('web_log.txt') # here we get the web log document into out spark context to manipulate as we want
  non_empty_lines = lines.filter(lambda line: len(line) > 0) #data cleaning: getting an rdd with all our info
  urls_accessed = non_empty_lines.map(lambda line: line.split(" ")[4]) #we want to build an rdd with url info taken from the initial rdd
  distinct_url_accessed = urls_accessed.distinct()  #again we want to create an rdd with only one occurence of the url
  for x in distinct_url_accessed.collect():
    print(x)
  sc.stop()
except:
  sc.stop()

In [ ]:
try:
  lines = sc.textFile('web_log.txt') # here we get the web log document into out spark context to manipulate as we want
  non_empty_lines = lines.filter(lambda line: len(line) > 0) #data cleaning: getting an rdd with all our info
  urls_accessed = non_empty_lines.map(lambda line: line.split(" ")[4]) #we want to build an rdd with url info taken from the initial rdd
  distinct_url_accessed = urls_accessed.distinct()  #again we want to create an rdd with only one occurence of the url
  URL = '/codemove/TTCENCUFMH3C'  # we have a variable that holds a specific value we want to 'search' in our data file 
  ip_url = non_empty_lines.filter(lambda line: line.split(" ")[4] == URL) # here we want to filter all the lines in our data file that do not hold our specific value to search
  for y in ip_url.collect(): #print our list of values, we will see that many ip adresses made a conection with a specific url that we gave
    print(y)
  sc.stop()
except:
  sc.stop()


2016-12-06T08:58:35.318+0000 37.139.9.11 404 GET /codemove/TTCENCUFMH3C 0.026  
2016-12-06T08:58:35.389+0000 37.139.9.11 404 GET /codemove/TTCENCUFMH3C 0.057  
2016-12-06T08:58:35.531+0000 37.139.9.11 404 GET /codemove/TTCENCUFMH3C 0.015  
2016-12-06T08:58:35.815+0000 37.139.9.11 404 GET /codemove/TTCENCUFMH3C 0.039  
2016-12-06T08:58:36.990+0000 37.139.9.11 404 GET /codemove/TTCENCUFMH3C 3.051  
2016-12-06T08:58:37.786+0000 37.139.9.11 404 GET /codemove/TTCENCUFMH3C 41.512  


In [ ]:
try:
  lines = sc.textFile('web_log.txt') # here we get the web log document into out spark context to manipulate as we want
  non_empty_lines = lines.filter(lambda line: len(line) > 0) #data cleaning: getting an rdd with all our info
  urls_accessed = non_empty_lines.map(lambda line: line.split(" ")[4]) #we want to build an rdd with url info taken from the initial rdd
  distinct_url_accessed = urls_accessed.distinct()  #again we want to create an rdd with only one occurence of the url
  ips = non_empty_lines.map(lambda line:line.split(" ")[1]) #data cleaning: our data has a lot of info but we want an rdd with only the ip adresses
  distinct_ips = ips.distinct() # data cleaning: since it is a web log entry we have a lot of duplication, distinct ips are only one occurence per ip
  
  answer = non_empty_lines.map(lambda line: line.split()).map(lambda v: (v[4] , {v[1]}))  #i want to get an inverted index with url and ips that accessed it, i will first get the combination mapping url to ip adress
  answer_final = answer.reduceByKey(lambda a,b: a | b)  #after getting previous rdd i will reduce by key having now distinct url and distinct ips that accessed it 

  URL = '/codemove/TTCENCUFMH3C'  # we have a variable that holds a specific value we want to 'search' in our data file 
  ip_url = non_empty_lines.filter(lambda line: line.split(" ")[4] == URL) # here we want to filter all the lines in our data file that do not hold our specific value to search
  for z,w in answer_final.collect(): #print our list of values, we will see that many ip adresses made a conection with a specific url that we gave
    print(z,w)
  sc.stop()
except:
  sc.stop()